In [ ]:
%pip install --upgrade pip 
%pip install -U langchain  
%pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph  
%pip install -qU "langchain[mistralai]" 
%pip install -qU langchain-huggingface
%pip install sentence-transformers
%pip install -qU langchain-core
%pip install -qU langchain_community pypdf pillow
%pip install hf_xet
%pip install -qU langchain-chroma

INDEXING

In [ ]:
os.environ["ANONYMIZED_TELEMETRY"] = "FALSE"
os.environ["CHROMA_TELEMETRY_ENABLED"] = "FALSE"
import getpass
import os
from langchain.chat_models import init_chat_model
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import PromptTemplate
from langchain_core.documents import Document
from typing import NamedTuple, Tuple
from langgraph.graph import START, StateGraph
from dotenv import load_dotenv,find_dotenv

In [ ]:
# Langsmith e os e getpass para lidar com as chaves de api do .env
load_dotenv(find_dotenv())  # Encontrar e carregar o documento .env que contém as chaves de api

os.environ["LANGSMITH_TRACING"] = "true"
if not os.environ.get("LANGSMITH_API_KEY"):
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter API key fot langsmith: ") # Se a chave de api não for encontrada o usuário deve inserir

In [ ]:
# Modelo de chat utilizado foi o mistral small, modelo open source da mistral ai
if not os.environ.get("MISTRAL_API_KEY"):
  os.environ["MISTRAL_API_KEY"] = getpass.getpass("Enter API key for Mistral AI: ") # Se a chave não for encontrada o usuário deve inserir

llm = init_chat_model("mistral-small-2503", model_provider = "mistralai") # Modelo de chat utilizado para a aplicação

In [ ]:
# Embedding model open-source do huggingface
embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-mpnet-base-v2")

In [ ]:
# Vector store chroma, database por diretório
vector_store = Chroma( 
    collection_name="exames_extraidos",
    embedding_function=embeddings,
    persist_directory="C:/JupyterNotebook/RAG/rag_chroma_db",  # Caminho do diretório que vai conter os embeddings
)

In [ ]:
# Carregando documentos em pdf por diretório
directory_path = (
    "C:/JupyterNotebook/RAG/RAG_exames"
)
loader = PyPDFDirectoryLoader("RAG_exames/")

docs = loader.load() # Carrega os documentos que o RAG vai usar

In [ ]:
# Dividindo os documento em chunks para fazer os embeddings
text_splitter = RecursiveCharacterTextSplitter(  # Divide os documentos em chunks para ser feito o embedding e guardar na base de dados
    chunk_size=1000,  # tamanho do chunk (caracteres)
    chunk_overlap=200,  # overlap do chunk (caracteres)
    add_start_index=True,  # acompanhar o indice do documento original
)
all_splits = text_splitter.split_documents(docs)

In [ ]:
# Storing documents
document_ids = vector_store.add_documents(documents=all_splits) # Coloca os documentos no vetor

INICIO DO RETRIEVAL E GENERATION

In [ ]:
# Prompt
template = """Você é um especialista em análise de relatórios de mamografia. Por favor, leia o relatório quando indicado e extraia as seguintes informações:
Cisto:
- Presente ou Ausente
- Localização e tamanho do cisto
Nódulo:
- Presente ou Ausente
- Localização e tamanho do nódulo
Calcificação:
- Presente ou Ausente
- Localização e tamanho da calcificação
Microcalcificação:
- Presente ou Ausente
- Localização e tamanho da microcalcificação
BI-RADS: [valor]
Outras citações a avaliar: [observações adicionais relevantes]

Caso não encontre alguma informação que se encaixe, coloque [sem referência no texto].

Diretrizes de Interpretação

Diferenciação entre Nódulo e Cisto:

Se um achado é identificado inicialmente como "nódulo" na mamografia, mas confirmado como "cisto" no ultrassom, classifique apenas como CISTO (presente).
Nódulos são estruturas sólidas; cistos são estruturas predominantemente líquidas.
Complexos sólido-císticos devem ser reportados em ambas categorias (nódulo E cisto).


Priorização de Achados Múltiplos:

Quando houver múltiplos cistos/nódulos, reporte TODOS, priorizando:
a) Achados classificados como suspeitos pelo relatório
b) Achados de maior tamanho
c) Achados com características atípicas mencionadas


Diferenciação entre Calcificações e Microcalcificações:

Calcificações: estruturas maiores, geralmente descritas como "grosseiras", "distróficas", "vasculares"
Microcalcificações: estruturas menores, frequentemente descritas como "puntiformes", "pleomórficas", "lineares", "agrupadas", "em cluster"
Se o relatório mencionar "microcalcificações", classifique especificamente como microcalcificações
Se mencionar apenas "calcificações", classifique como calcificações.

{context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)

In [ ]:
# estado para o langgraph
class State(NamedTuple):  # Informações que o pipeline vai carregar de um nó para o outro. Facilita leitura, manutenção e evita bugs
    question: str  # Pergunta feita pelo usuário
    context: Tuple[Document, ...] # Contexto recuperado da base de dados
    answer: str  # Resposta gerada pela llm

In [ ]:
# Funções do rag
def retrieve(state: State):  # Recupera nos documentos dados parecidos com os inputados
    retrieved_docs = vector_store.similarity_search(state.question) # Faz uma comparação da pergunta com os dados no vetor e recupera os mais parecidos
    return State(
        question=state.question, 
        context=tuple(retrieved_docs), # Usamos tuple em vez de lista por ser um estrutura imutável, o que facilita o uso do LangGraph
        answer=state.answer
    )

def generate(state: State):  # Gera uma resposta com a adição das informações encontradas no vetor
    docs_content = "\n\n".join(doc.page_content for doc in state.context)
    messages = custom_rag_prompt.invoke({"question": state.question, "context": docs_content})
    response = llm.invoke(messages)
    return State(
        question=state.question,
        context=state.context,
        answer=response.content
    )

In [ ]:
# O LangGraph vai garantir que cada etapa (nó) do fluxo receba e devolva um objeto do tipo State
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [ ]:
state_inicial = State(  # Estado incial da aplicação necessário por causa do uso de tuple
    question="""Faça a extração das características do seguinte exame mamográfico.

MAMOGRAFIA DIGITAL DR* BILATERAL

Indicação clínica: 69 anos. Rotina. Antecedente de neoplasia mamária.

Exame com MAMÓGRAFO DIGITAL nas incidências craniocaudal e mediolateral oblíqua acrescido de incidências em ambas projeções obtidas com manobras de deslocamento posterior dos implantes mamários.

Status pós cirurgia conservadora da mama esquerda.
Parênquima mamário heterogeneamente denso, o que reduz a sensibilidade da mamografia.
Alterações arquiteturais, relacionadas à mamoplastia.
Nódulo denso de contornos espiculados projetado no QSE da mama esquerda, associado a retração cutânea,
com correspondência ao ultrassom, maior em relação ao exame de 01/2024. Prosseguir com core biopsy.
Cisto oleoso na mama esquerda.
Calcificações esparsas.
Ausência de microcalcificações pleomórficas agrupadas ou ramificadas.
Implante bilateral, sem sinais de roturas extracapsulares.
Linfonodo axilar, de aspecto reacional.

ACR-BIRADS® categoria 5.""",
    context=tuple([]),
    answer=" "
)

response = graph.invoke(state_inicial)
print(response["answer"])

In [ ]:
# Contexto recuperado da base de dados do RAG
print(f'Context: {response["context"]}\n\n')

In [ ]:
# LangGraph pra acompanhar o RAG
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))